데이터셋은 우선적으로는 CelebDFV2를 사용하고, 이후에 GTA-V 데이터를 합쳐보도록 하자.

이걸 어디부터 구현해야 할까...

일단 SigLIP을 불러오는 것부터 시작하자. Quantize를 해줘서 최대한 부담을 줄여주자.

In [1]:
!uv pip install --system lightning
!pip install -q lightning

Using Python 3.12.12 environment at: /usr
Resolved 43 packages in 327ms
Prepared 4 packages in 87ms
Installed 4 packages in 14ms
 + lightning==2.6.0
 + lightning-utilities==0.15.2
 + pytorch-lightning==2.6.0
 + torchmetrics==1.8.2


In [2]:
!uv pip install --system bitsandbytes
!pip install -q bitsandbytes

Using Python 3.12.12 environment at: /usr
Resolved 29 packages in 32ms
Prepared 1 package in 731ms
Installed 1 package in 3ms
 + bitsandbytes==0.49.1


In [16]:
!uv pip install --system decord
!pip install -q decord

Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 31ms
Prepared 1 package in 146ms
Installed 1 package in 3ms
 + decord==0.6.0


In [3]:
import lightning.pytorch as L
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModel, BitsAndBytesConfig

embed_size = 768

frame_token = 576

In [4]:
torch.cuda.empty_cache()
import gc
gc.collect()

90

솔직히, 왠만하면 원본 모델을 따라가고 싶지만... 일단 인코더를 더 경량으로 바꾼다.

나중에 성능 안 나오면 탓할 것 중 인코더가 늘었다.

In [4]:
import torch.nn as nn

In [5]:
import torch.nn.functional as F

class ViTEncoder(nn.Module):
    def __init__(self, embed_size=768, num_heads=12, dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = embed_size // num_heads
        assert self.head_dim * num_heads == embed_size, "embed_size must be divisible by num_heads"

        self.q_proj = nn.Linear(embed_size, embed_size)
        self.k_proj = nn.Linear(embed_size, embed_size)
        self.v_proj = nn.Linear(embed_size, embed_size)
        self.out_proj = nn.Linear(embed_size, embed_size)
        self.dropout_p = dropout

        self.ln1 = nn.LayerNorm(embed_size)
        self.mlp = nn.Sequential(
            nn.Linear(embed_size, 4 * embed_size),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(4 * embed_size, embed_size),
            nn.Dropout(dropout),
        )
        self.ln2 = nn.LayerNorm(embed_size)

    def forward(self, x, return_head_contrib=False):
        # x: (batch_size, seq_len, embed_size)
        batch_size, seq_len, embed_size = x.shape

        # Project queries, keys, values
        q = self.q_proj(x) # (batch_size, seq_len, embed_size)
        k = self.k_proj(x) # (batch_size, seq_len, embed_size)
        v = self.v_proj(x) # (batch_size, seq_len, embed_size)

        # Split into multiple heads
        q = q.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2) # (b, num_heads, seq_len, head_dim)
        k = k.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2) # (b, num_heads, seq_len, head_dim)
        v = v.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2) # (b, num_heads, seq_len, head_dim)

        # Apply scaled dot product attention
        # dropout_p는 훈련 중일 때만 적용
        attn_output_raw = F.scaled_dot_product_attention(
            q, k, v,
            attn_mask=None,
            dropout_p=self.dropout_p if self.training else 0.0,
            is_causal=False
        )
        # attn_output: (b, num_heads, seq_len, head_dim)


        head_contrib = None
        if return_head_contrib:
            # out_proj_weight: [embed_size, embed_size] -> [num_heads, head_dim, embed_size]
            out_proj_weight = self.out_proj.weight.t().view(self.num_heads, self.head_dim, embed_size)
            # b: batch size, h: num_head, l: seq_len, k: head_dim, d: embed_dim
            head_contrib = torch.einsum("bhlk, hkd -> bhld", attn_output_raw, out_proj_weight)
            # -> result: [batch, num_head, seq_len, embed_dim] (-> [batch, num_head, seq_len, embed_dim])
            # Have to remove class token, and pool that with frame dimension

        # Concatenate heads and apply final linear projection
        attn_output = attn_output_raw.transpose(1, 2).contiguous().view(batch_size, seq_len, embed_size)
        attn_output = self.out_proj(attn_output) # (batch_size, seq_len, embed_size)


        x = self.ln1(x + attn_output) # Residual connection + LayerNorm
        x = self.ln2(x + self.mlp(x))

        if return_head_contrib:
            return x, head_contrib
        return x

In [6]:
import math
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float=0.1, max_len: int=32): # max_len을 32로 설정
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        # d_model에 맞춰 div_term 계산
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))

        # [max_len, d_model] 형상으로 생성
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # [1, max_len, d_model]로 변경하여 Batch First 대응
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: [batch_size, frame * seq_len, d_model]
        """
        b, f, t, d = x.shape
        # [1, max_len, 1, d_model]로 -> frame과 d_model에만 적용한다
        # seq_len의 공간적 위치 정보는 이미 인코더에서 처리해주었다.
        curr_pe = self.pe[:, :f, :].unsqueeze(2)
        # 입력된 x의 길이만큼만 PE를 더함
        x = x + curr_pe
        return self.dropout(x)

In [7]:
import torch
from torchvision.transforms import v2

class GPUSigLIPProcessor:
    def __init__(self, processor):
        config = processor.image_processor

        # 1. 리사이즈 설정: Bilinear + Antialias=True가 핵심
        # Fast 프로세서가 텐서를 처리할 때 사용하는 로직과 일치시킵니다.
        self.resize = v2.Resize(
            size=(config.size['height'], config.size['width']),
            interpolation=v2.InterpolationMode.BILINEAR, # resample=2
            antialias=True # 오차를 줄이는 가장 중요한 설정
        )

        # 2. 정규화 설정
        # (x - 0.5) / 0.5 연산
        self.mean = torch.tensor(config.image_mean).view(1, 3, 1, 1)
        self.std = torch.tensor(config.image_std).view(1, 3, 1, 1)
        self.rescale_factor = config.rescale_factor

    def __call__(self, video_tensor):
        """
        video_tensor: (B, 3, T, H, W), uint8, GPU
        """
        b, c, t, h, w = video_tensor.shape
        device = video_tensor.device

        # 차원 변경 (B*T, C, H, W)
        x = video_tensor.permute(0, 2, 1, 3, 4)
        x = x.flatten(0, 1)

        # [Step 1] Resize (uint8 상태에서 수행하거나 float32에서 수행)
        # torchvision v2는 uint8 입력을 받아 내부적으로 고정밀 연산을 수행합니다.
        x = self.resize(x)

        # [Step 2] Float32 변환 및 Rescale (0~255 -> 0~1)
        x = x.to(torch.float32) * self.rescale_factor

        # [Step 3] Normalize (x - 0.5) / 0.5
        # mean, std를 캐싱하여 속도 최적화
        self.mean = self.mean.to(device)
        self.std = self.std.to(device)
        x = (x - self.mean) / self.std

        # [Step 4] 최종 모델 입력형태인 float16으로 반환
        return x.to(torch.float16)

In [8]:
class UNITE(nn.Module):
    def __init__(self, num_channel=3, num_cls=2, num_heads=12, max_len=32, dropout=0.1):
        super().__init__()

        model_id = "google/siglip2-base-patch16-384"
        self.vis_encoder = AutoModel.from_pretrained(
            model_id,
            device_map="auto",
            dtype=torch.bfloat16,
            attn_implementation="sdpa",
        )
        self.embed_size = self.vis_encoder.config.vision_config.hidden_size
        processor = AutoProcessor.from_pretrained(model_id, use_fast=True)
        self.processor = GPUSigLIPProcessor(processor)
        self.num_heads = num_heads

        for para in self.vis_encoder.parameters():
            para.requires_grad = False
        self.vis_encoder.eval()

        self.class_token = nn.Parameter(torch.randn((self.embed_size,)), requires_grad=True)

        self.pos_embedding = PositionalEncoding(self.embed_size)
        self.first_encoder = ViTEncoder(self.embed_size, num_heads, dropout)
        self.encoders = nn.ModuleList([ViTEncoder(self.embed_size, num_heads, dropout) for _ in range(3)])
        self.mlp_head = nn.Linear(self.embed_size, num_cls)


    def forward(self, x, return_ad_param=False):
        self.vis_encoder.eval()

        # Input: [batch, c, frame, h, w]
        b, _, f, *_ = x.shape

        with torch.no_grad():
            # -> Preprocessing [batch * frame, c, h, w]
            x = self.processor(x)
            # -> Visual encoding [batch * frame, token/frame(576), dim/token (embed_size)]
            x = self.vis_encoder.vision_model(pixel_values=x).last_hidden_state
        # -> [batch, frame, token/frame, dim/token]
        x = x.reshape(b, f, -1, self.embed_size)
        x = self.pos_embedding(x)
        train_in = x # xi

        _, _, t, d = x.shape
        # Reshape for transformer
        # -> [batch, total token, dim/token]
        x = x.reshape(b, f*t, d)
        # Add class token
        cls_token = self.class_token.view(1, 1, -1).expand(b, -1, -1)
        x = torch.cat([cls_token, x], dim=1)
        P = None
        if return_ad_param:
            x, head_contrib = self.first_encoder(x, return_head_contrib=True)
            # head_contrib: [batch, head, total token + class token, dim/token]
            # -> [batch, head, total token, dim/token]
            head_contrib = head_contrib[:, :, 1:, :]
            # -> [batch, head, frame, token/frame, dim/token]
            head_contrib = head_contrib.view(b, self.num_heads, f, t, d)
            # -> mean pooling [batch, head, token/frame, dim/token]
            head_contrib = head_contrib.mean(dim=2) # A

            P = torch.einsum("bftd, bhtd -> bhf", train_in, head_contrib)
        else:
            x = self.first_encoder(x)

        for encoder in self.encoders:
            x = encoder(x)

        # Get only cls_token
        x = x[:, 0, :]
        x = x.view(b, -1)
        x = self.mlp_head(x)
        if return_ad_param:
            return x, P
        return x

In [9]:
class ADLoss(nn.Module):
    def __init__(self, num_cls=2, num_heads=12, max_len=32, delta_within=(0.01, -2.0), delta_between=0.5, eta=0.05):
        super().__init__()
        # C shape: [num_classes, num_heads, max_len]
        # 논문 식(3)에 따라 센터를 각 클래스별로 유지해야 함
        C = torch.zeros(num_cls, num_heads, max_len)
        self.register_buffer('C', C)

        self.num_cls = num_cls
        self.delta_within = torch.tensor(delta_within) # [0.01, -2.0] (True, Fake)
        self.delta_between = delta_between # 0.5
        self.eta = eta

    def forward(self, P, labels, log_detail=False):
        """
        P: [batch, num_heads, max_len] (Pooled Features)
        labels: [batch] (Class indices)
        """
        device = P.device
        self.delta_within = self.delta_within.to(device)

        P_norm = F.normalize(P.view(P.size(0), -1), p=2, dim=1).view_as(P)
        C_norm = F.normalize(self.C.view(self.num_cls, -1), p=2, dim=1).view_as(self.C)

        # --- 1. 센터 업데이트 (식 3) ---
        # 배치의 각 클래스별 평균을 구해서 업데이트
        for c in range(self.num_cls):
            mask = (labels == c)
            if mask.any():
                # 해당 클래스의 이번 배치 평균
                batch_class_mean = P_norm[mask].mean(dim=0) # [num_heads, max_len]
                # print(f"{self.C[c].shape, P.shape, P[mask].shape, batch_class_mean.shape=}")
                # 이동 평균 업데이트
                with torch.no_grad():
                    self.C[c] = (1 - self.eta) * self.C[c] + self.eta * batch_class_mean.detach()

        # --- 2. Within-class Loss (식 4) ---
        # 각 샘플과 자기 클래스 센터 사이의 거리
        # P: [B, H, F], self.C[labels]: [B, H, F]
        diff_within = P_norm - C_norm[labels]
        # L2 Norm 계산 (헤드와 프레임 차원에 대해)
        dist_within = torch.norm(diff_within, p=2, dim=(1, 2))

        # 각 샘플별 delta 적용
        loss_within = torch.relu(dist_within - self.delta_within[labels]).mean()

        # --- 3. Between-class Loss (식 5: 서로 다른 쌍에 대해 전부) ---
        # 클래스 센터들 간의 모든 쌍 거리 (Pairwise Distance) 계산
        # self.C: [num_classes, H*F]로 펼쳐서 계산하면 편리함
        C_flat = C_norm.view(self.num_cls, -1)

        # 모든 클래스 쌍 간의 차이 계산: [num_classes, num_classes, H*F]
        # Broadcasting 활용: (N, 1, D) - (1, N, D) -> (N, N, D)
        diff_between = C_flat.unsqueeze(1) - C_flat.unsqueeze(0)

        # 모든 쌍의 거리 행렬: [num_classes, num_classes]
        dist_matrix = torch.norm(diff_between, p=2, dim=2)

        # k != l 인 조건 (서로 다른 쌍) 추출을 위한 마스크
        # torch.triu를 써서 중복 계산(k,l과 l,k)을 피하고 자기 자신(k=l)도 제외함
        mask_between = torch.triu(torch.ones(self.num_cls, self.num_cls, device=device), diagonal=1).bool()

        # 서로 다른 클래스 쌍의 거리들만 추출
        different_pairs_dist = dist_matrix[mask_between]

        # 식 (5) 적용: max(delta - dist, 0)
        loss_between = torch.relu(self.delta_between - different_pairs_dist).sum()
        if log_detail:
            return loss_within + loss_between, loss_within, loss_between

        return loss_within + loss_between

In [10]:
from torchmetrics.classification import Accuracy, AveragePrecision, Precision, Recall

In [11]:
class LitUNITEClassifier(L.LightningModule):
    def __init__(
            self, num_cls=2, num_heads=12, max_len=32, dropout=0.1,
            delta_within=(0.01, -2.0), delta_between=0.5, eta=0.05,
            lambda_1=0.5, lambda_2=0.5, lr=1e-4, decay_steps=1000,
        ):
        super().__init__()
        self.save_hyperparameters()
        self.model = UNITE(
            num_cls=num_cls,
            num_heads=num_heads,
            max_len=max_len,
            dropout=dropout,
        )
        self.ce_loss = nn.CrossEntropyLoss()
        self.ad_loss = ADLoss(
            num_cls=num_cls,
            num_heads=num_heads,
            max_len=max_len,
            delta_within=delta_within,
            delta_between=delta_between,
            eta=eta,
        )
        self.lambda_1 = lambda_1
        self.lambda_2 = lambda_2

        self.lr = lr
        self.decay_steps = decay_steps # Set this in respect to batch size; original batch size was 32

        self.acc = Accuracy(task='multiclass', num_classes=num_cls)
        self.ap = AveragePrecision(task='multiclass', num_classes=num_cls)
        self.precision = Precision(task='multiclass', num_classes=num_cls)
        self.recall = Recall(task='multiclass', num_classes=num_cls)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logit, P = self.model(x, return_ad_param=True)
        loss_ad, within, between = self.ad_loss(P, y, log_detail=True)
        loss_ce = self.ce_loss(logit, y)
        loss = loss_ce * self.lambda_1 + loss_ad * self.lambda_2
        self.log("train/loss_ad", loss_ad, logger=True)
        self.log("train/loss_ad/loss_within", within, logger=True)
        self.log("train/loss_ad/loss_between", between, logger=True)
        self.log("train/loss_ce", loss_ce, logger=True)
        self.log("train/loss", loss, prog_bar=True, logger=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logit, P = self.model(x, return_ad_param=True)
        loss_ad = self.ad_loss(P, y)
        loss_ce = self.ce_loss(logit, y)
        loss = loss_ce * self.lambda_1 + loss_ad * self.lambda_2
        self.log("val/loss_ad", loss_ad, logger=True)
        self.log("val/loss_ce", loss_ce, logger=True)
        self.log("val/loss", loss, prog_bar=True, logger=True)

        self.acc(logit, y)
        self.ap(logit, y)
        self.precision(logit, y)
        self.recall(logit, y)
        self.log("val/acc", self.acc, logger=True)
        self.log("val/ap", self.ap, logger=True)
        self.log("val/precision", self.precision, logger=True)
        self.log("val/recall", self.recall, logger=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logit = self.model(x)

        self.acc(logit, y)
        self.ap(logit, y)
        self.precision(logit, y)
        self.recall(logit, y)
        self.log("test/acc", self.acc, logger=True)
        self.log("test/ap", self.ap, logger=True)
        self.log("test/precision", self.precision, logger=True)
        self.log("test/recall", self.recall, logger=True)

    def configure_optimizers(self):
        optim = torch.optim.AdamW(self.model.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.StepLR(optim, self.decay_steps, gamma=0.5)

        return {
            "optimizer": optim,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",
            },
        }

In [17]:
import kagglehub
from pathlib import Path
import cv2
import pandas as pd
import torch
import numpy as np
import math

from decord import VideoReader, cpu
from torch.utils.data import DataLoader, Dataset, random_split


class CelebDFDataset(Dataset):
    def __init__(self, is_test: bool, path_str:str, length=32, size=(384, 384), transform=None):
        """
        Args:
            is_test (bool): True면 테스트 셋, False면 트레인 셋 로드
            length (int): 시퀀스 길이 (기본 32)
            transform: 이미지 전처리 (Optional)
        """

        self.path = Path(path_str)
        self.is_test = is_test
        self.length = length
        self.size = size
        self.transform = transform

        # 모든 mp4 파일 검색
        self.files = list(self.path.glob("*/*.mp4"))

        # 테스트 비디오 리스트 로드
        # 파일 형식: [1|0] [path] (예: 1 YouTube-real/00170.mp4)
        txt_path = self.path / "List_of_testing_videos.txt"
        test_df = pd.read_csv(txt_path, sep=" ", header=None, names=["label", "path"])

        # 비교를 위해 테스트 파일 경로들을 Set으로 변환 (검색 속도 향상)
        # Windows/Linux 경로 구분자 통일을 위해 '/'로 replace 처리
        test_files_set = set(test_df["path"].apply(lambda x: x.replace("\\", "/")).values)

        self.samples = [] # (video_path, chunk_index, label) 튜플을 저장할 리스트

        print(f"Processing metadata for {'Test' if is_test else 'Train'} set...")

        for next_file in self.files:
            # 데이터셋 루트 기준 상대 경로 (예: YouTube-real/00170.mp4)
            rel_path = str(next_file.relative_to(self.path)).replace("\\", "/")

            # 현재 파일이 테스트 리스트에 있는지 확인
            is_in_test_list = rel_path in test_files_set

            # 요청한 Split(Train/Test)과 맞지 않으면 스킵
            if self.is_test != is_in_test_list:
                continue

            # 레이블 결정 (0: Real, 1: Fake)
            # 폴더명 기반 판단
            if "YouTube-real" in rel_path or "Celeb-real" in rel_path:
                label = 0
            elif "Celeb-synthesis" in rel_path:
                label = 1
            else:
                continue # 알 수 없는 폴더는 제외

            cap = cv2.VideoCapture(str(next_file))
            if cap.isOpened():
                frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                cap.release()

                if frame_cnt <= 0: continue

                # 영상 하나를 여러 데이터(Chunk)로 쪼개기
                # Stride=2 (하나 걸러 하나), Length=32
                # 데이터 하나당 필요한 원본 프레임 구간 = 약 64 프레임
                # 전체 프레임에서 Stride 2로 뽑았을 때 나오는 유효 프레임 수
                effective_frames = math.ceil(frame_cnt / 2)

                # 영상 하나에서 나오는 데이터 개수 (올림 처리)
                num_chunks = math.ceil(effective_frames / self.length)

                for i in range(num_chunks):
                    self.samples.append({
                        "video_path": str(next_file),
                        "chunk_idx": i,
                        "label": label
                    })
            else:
                print(f"Cannot open video: {next_file}")

        print(f"Loaded {len(self.samples)} samples from {len(self.files)} files.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        meta = self.samples[idx]
        video_path = meta["video_path"]
        label = meta["label"]
        chunk_idx = meta["chunk_idx"]

        try:
            # 1. VideoReader 객체 생성 (CPU 컨텍스트 사용)
            # num_threads=0은 가용한 모든 코어를 사용하여 디코딩함을 의미합니다.
            # vr = VideoReader(video_path, ctx=cpu(0), num_threads=0, width=self.size[0], height=self.size[1])
            vr = VideoReader(video_path, ctx=cpu(0), num_threads=0)
            total_frames = len(vr)

            # 2. 가져올 프레임 인덱스 계산 (Stride 2 적용)
            start_frame = chunk_idx * self.length * 2
            indices = []
            for i in range(self.length):
                idx_to_fetch = start_frame + (i * 2)
                # 영상 길이를 넘어갈 경우 마지막 프레임으로 패딩
                if idx_to_fetch < total_frames:
                    indices.append(idx_to_fetch)
                else:
                    indices.append(total_frames - 1)

            # 3. 한 번에 여러 프레임 가져오기 (가장 핵심적인 속도 향상 포인트)
            # get_batch 결과는 [T, H, W, C] 형태의 decord.NDArray입니다.
            frames = vr.get_batch(indices).asnumpy()

        except Exception as e:
            print(f"Error loading {video_path}: {e}")
            # 에러 발생 시 0으로 채워진 텐서 반환 (학습 중단 방지)
            frames = np.zeros((self.length, self.size[1], self.size[0], 3), dtype=np.uint8)

        # 4. Resize 처리 (Batch 단위 처리를 위해 루프 사용)
        # 만약 torchvision v2를 사용한다면 텐서 상태에서 배치로 resize 하는 것이 더 빠릅니다.
        # resized_frames = []
        # for frame in frames:
            # resized_frames.append(cv2.resize(frame, self.size))

        # frames_np = np.array(resized_frames)
        frames_np = np.array(frames)

        # 5. Tensor 변환 및 차원 변경: (T, H, W, C) -> (C, T, H, W)
        frames_tensor = torch.from_numpy(frames_np).permute(3, 0, 1, 2)

        return frames_tensor.contiguous(), torch.tensor(label)

In [24]:
import subprocess
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from shutil import copy2
import os

def resize_single_video(item):
    src_path, dst_path, size = item
    # 폴더가 없으면 생성
    dst_path.parent.mkdir(parents=True, exist_ok=True)

    # FFmpeg 명령어 구성
    # -y: 기존 파일 덮어쓰기
    # -i: 입력 파일
    # -vf: 비디오 필터 (리사이즈)
    # -c:v libx264: H.264 코덱 사용
    # -crf 23: 일반적인 화질 설정 (낮을수록 고화질)
    # -preset veryfast: 속도 우선 인코딩
    # -an: 오디오 제거 (학습에 필요 없음, 용량 절감)
    cmd = [
        'ffmpeg', '-y', '-i', str(src_path),
        '-vf', f'scale={size[0]}:{size[1]}',
        '-c:v', 'libx264', '-crf', '18', '-preset', 'medium',
        '-an', str(dst_path)
    ]

    # 실행 (로그는 숨김)
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

def preprocess_celebdf(src_root, dst_root, size=(384, 384), max_workers=8):
    src_root = Path(src_root)
    dst_root = Path(dst_root)

    if dst_root.exists():
        print(f"Target directory {dst_root} already exists. Skipping preprocessing.")
        return

    # 모든 mp4 파일 찾기
    video_files = list(src_root.glob("*/*.mp4"))
    tasks = []

    for src_path in video_files:
        rel_path = src_path.relative_to(src_root)
        dst_path = dst_root / rel_path
        if not dst_path.exists():
            tasks.append((src_path, dst_path, size))

    print(f"Starting preprocessing: {len(tasks)} videos...")

    # 멀티프로세싱으로 병렬 처리
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(resize_single_video, tasks), total=len(tasks)))
    copy2(src_root / "List_of_testing_videos.txt", dst_root / "List_of_testing_videos.txt")

    print("Preprocessing completed.")

In [20]:
class CelebDFDataModule(L.LightningDataModule):
    def __init__(self, length=32, batch_size=32, num_workers=8):
        super().__init__()
        self.length = length
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.preprocessed_path = "/content/preprocessed"

    def prepare_data(self):
        path = kagglehub.dataset_download("reubensuju/celeb-df-v2")
        preprocess_celebdf(path, self.preprocessed_path, max_workers=12)

    def setup(self, stage=None):
        path = self.preprocessed_path
        if stage == "fit" or stage is None:
            train_full = CelebDFDataset(is_test=False, path_str=path)
            self.celebdf_train, self.celebdf_val = random_split(train_full, [0.9, 0.1])
        if stage == "test" or stage is None:
            self.celebdf_test = CelebDFDataset(is_test=True, path_str=path)

    def train_dataloader(self):
        return DataLoader(self.celebdf_train, num_workers=self.num_workers, batch_size=self.batch_size, pin_memory=True, persistent_workers=True)
    def val_dataloader(self):
        return DataLoader(self.celebdf_val, num_workers=self.num_workers, batch_size=self.batch_size, pin_memory=True, persistent_workers=True)
    def test_dataloader(self):
        return DataLoader(self.celebdf_test, num_workers=self.num_workers, batch_size=self.batch_size, shuffle=False)

In [1]:
import wandb
from google.colab import userdata
wandb_key = userdata.get('wandb_api')
wandb.login(key=wandb_key)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dhnam0502 (dhnam0502-likelion) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [21]:
BATCH_SIZE = 20
DECAY_STEPS = (1000 * 32) // BATCH_SIZE

datamodule = CelebDFDataModule(batch_size=BATCH_SIZE)
lit_classifier = LitUNITEClassifier(decay_steps=DECAY_STEPS)
# lit_classifier = torch.compile(lit_classifier)

In [ ]:
datamodule.prepare_data()

Using Colab cache for faster access to the 'celeb-df-v2' dataset.
Starting preprocessing: 6529 videos...


  0%|          | 30/6529 [00:29<1:46:27,  1.02it/s]


In [71]:
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint
from lightning.pytorch.callbacks.lr_monitor import LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger
from lightning.pytorch.callbacks import TQDMProgressBar


wandb_logger = WandbLogger(project="UNITE_deepfake_classification", name="baseline_profile", log_model=True)

ckpt = ModelCheckpoint(monitor="val/acc", mode="max", save_last=True)
lr_monitor = LearningRateMonitor(logging_interval='epoch')

trainer =  L.Trainer(
    # max_epochs=25,
    max_steps=10,
    profiler='simple',
    logger=wandb_logger,
    callbacks=[ckpt, lr_monitor, TQDMProgressBar()],
    precision='bf16-mixed'
)



INFO:pytorch_lightning.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores


In [2]:
wandb.finish()

In [ ]:
trainer.fit(lit_classifier, datamodule=datamodule)